# Model Training

In [5]:
import pandas as pd
import numpy as np

In [6]:
diamond_df = pd.read_csv("data/gemstone.csv")
diamond_df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [7]:
diamond_df.drop('id', axis=1)
diamond_df.head()


,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


### Getting Independent and Dependent features

In [8]:
X = diamond_df.drop('price',axis=1)
y= diamond_df['price']

X.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [9]:
y.head()

0    13619
1    13387
2     2772
3      666
4    14453
Name: price, dtype: int64

### Segregating Numerical and Categorical features

In [10]:
numerical_columns = X.select_dtypes(exclude='object').columns
categorical_columns = X.select_dtypes(include='object').columns

print("Numerical columns : ",numerical_columns)
print("Catgorical columns : ",categorical_columns)

Numerical columns :  Index(['id', 'carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')
Catgorical columns :  Index(['cut', 'color', 'clarity'], dtype='object')


#### Define the custom ranking for each ordinal variable

In [11]:

cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
# Numerical Pipeline
num_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())

])

# Categorical Pipeline
cat_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal_encoder', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler', StandardScaler())
    ]
)

In [14]:
# Preprocessor
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_columns),
    ('cat_pipeline', cat_pipeline, categorical_columns)
])

#### train test split

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=123)



### Data Transformation

In [16]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())



In [17]:
X_train.head()

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.758990,-0.822482,0.719310,-0.639531,-0.851215,-0.897134,-0.817000,0.875291,-0.937424,0.678236
1,0.882918,-1.060421,0.626993,-0.117901,-1.329029,-1.305356,-1.266265,-1.135438,0.295870,-1.317038
2,-0.954516,-0.887374,-0.019221,-1.682792,-0.950384,-0.933420,-0.932939,0.875291,-0.937424,0.013145
3,0.982747,2.854766,-0.757751,1.446990,2.322190,2.368641,2.211914,-0.130073,-0.320777,-1.317038
4,-0.193654,-1.038791,-0.296170,-1.161161,-1.202814,-1.196497,-1.208295,0.875291,-0.937424,-0.651947


In [18]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

### Simple Linear Regression Model

In [19]:
linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train, y_train)

LinearRegression()

In [20]:
linear_regression_model.intercept_

3968.730968775142

In [21]:
linear_regression_model.coef_

array([ 2.54861559e+00,  6.46085626e+03, -1.31552364e+02, -6.75143402e+01,
       -2.00036520e+03, -2.22240600e+02, -8.54263894e+01,  7.16008845e+01,
       -4.65737144e+02,  6.51338487e+02])

### Evaluate Model Function

In [22]:
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2_square = r2_score(y_true, y_pred)
    return mae, rmse, r2_square

### Model Training

In [24]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso' : Lasso(),
    'Ridge' : Ridge(),
    'ElasticNet' : ElasticNet(),
    'RandomForestRegressor' : RandomForestRegressor(),
    'LightGBMRegressor' : LGBMRegressor(),
    'XGBoostRegressor' : XGBRegressor()
}

model_list = []
trained_model_list = []
r2_list = []


for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae, rmse, r2_square = evaluate_model(y_test, y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1009.9434754842646
MAE: 674.8176220844555
R2 score 93.72329751377869


Lasso
Model Training Performance
RMSE: 1010.0070203103692
MAE: 675.9483902554992
R2 score 93.72250763884925


Ridge
Model Training Performance
RMSE: 1009.9277756818659
MAE: 674.8421989018759
R2 score 93.72349265781438


ElasticNet
Model Training Performance
RMSE: 1527.9904174016124
MAE: 1063.2700714266925
R2 score 85.63258391592625


RandomForestRegressor
Model Training Performance
RMSE: 591.8839897181143
MAE: 306.25049008127843
R2 score 97.84418944878149


LightGBMRegressor
Model Training Performance
RMSE: 569.2735852069916
MAE: 298.4567168345245
R2 score 98.00575058537963


XGBoostRegressor
Model Training Performance
RMSE: 579.6858096992479
MAE: 300.0925641515101
R2 score 97.93213231465452




In [25]:
model_list

['LinearRegression',
 'Lasso',
 'Ridge',
 'ElasticNet',
 'RandomForestRegressor',
 'LightGBMRegressor',
 'XGBoostRegressor']